# Mortgage Document Extraction

---

At this point we have identified the documents, and we can now start extracting information from them. What we want to extract information from each document will depend on the type of document as depicted in the figure below.

<p align="center">
  <img src="./images/extraction.png" alt="exn" width="800px"/>
</p>



---

## Setup Notebook

We will be using the [Amazon Textract Parser Library](https://github.com/aws-samples/amazon-textract-response-parser/tree/master/src-python) for parsing through the Textract response, data science library [Pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) for content analysis, the [Amazon SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/), and [AWS boto3 python sdk](https://boto3.amazonaws.com/v1/documentation/api/latest/index.html) to work with Amazon Textract and Amazon A2I. Let's now install and import them.

In [2]:
!python -m pip install -q amazon-textract-response-parser --upgrade --force-reinstall
!python -m pip install -q amazon-textract-caller --upgrade --force-reinstall
!python -m pip install -q amazon-textract-prettyprinter --upgrade --force-reinstall
!python -m pip install pdfplumber
!sed -i '/disable ghostscript format types/,+6d' /etc/ImageMagick-6/policy.xml

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
amazon-textract-prettyprinter 0.1.10 requires amazon-textract-response-parser<0.2,>=0.1, but you have amazon-textract-response-parser 1.0.3 which is incompatible.
awscli 1.32.84 requires botocore==1.34.84, but you have botocore 1.35.28 which is incompatible.
sagemaker-datawrangler 0.4.3 requires sagemaker-data-insights==0.4.0, but you have sagemaker-data-insights 0.3.3 which is incompatible.
sphinx 7.2.6 requires docutils<0.21,>=0.18.1, but you have docutils 0.16 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
amazon-textract-prettyprinter 0.1.10 requires amazon-textract-response-parser<0.2,>=0.1, but you have amazon-textract-response-parser 1.0.3 which is incompatib

In [3]:
#Restart the kernel
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [1]:
import boto3
import botocore
import sagemaker
import os
import io
import datetime
import json
import pandas as pd
from PIL import Image as PImage, ImageDraw
from pathlib import Path
from IPython.display import Image, display, HTML, JSON, IFrame
from textractcaller import call_textract_lending
from textractprettyprinter.t_pretty_print import convert_lending_from_trp2
import trp.trp2_lending as tl
import multiprocessing as mp
from trp import Document
from sagemaker import get_execution_role


# variables
data_bucket = sagemaker.Session().default_bucket()
region = boto3.session.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

os.environ["BUCKET"] = data_bucket
os.environ["REGION"] = region
role = sagemaker.get_execution_role()

print(f"SageMaker role is: {role}\nDefault SageMaker Bucket: s3://{data_bucket}")

s3=boto3.client('s3')
textract = boto3.client('textract', region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
SageMaker role is: arn:aws:iam::463470940958:role/service-role/AmazonSageMaker-ExecutionRole-20240909T192441
Default SageMaker Bucket: s3://sagemaker-eu-west-2-463470940958


In [2]:
documentName = "docs/lending_package.pdf"
display(IFrame(documentName, 500, 600));

---
# Extracting information from documents


From module 1 - Document classification, we have the following as documents identified from the lending packet. We will extract the information out of each of the Mortgage documents

- Page Number: 1 , Page Classification: `PAYSLIPS`
- Page Number: 2 , Page Classification: `CHECKS`
- Page Number: 3 , Page Classification: `IDENTITY_DOCUMENT`
- Page Number: 4 , Page Classification: `1099_DIV`
- Page Number: 5 , Page Classification: `BANK_STATEMENT`
- Page Number: 6 , Page Classification: `W_2`
- Page Number: 7 , Page Classification: `UNCLASSIFIED`
    
    
We will use Amazon Textract's [Analyze Lending API](https://docs.aws.amazon.com/textract/latest/dg/analyzing-document-text.html) API to extract Key-Value pairs out of the documents. Note that, in some cases (for example; for the sample) the API will also detect the presence of [signatures.](https://docs.aws.amazon.com/textract/latest/dg/lending-response-objects.html)

For the `Unclassified` type of document,i.e., Page Number 7 (which is a sample Homeowners insurance application form), we will use Amazon Textract Queries feature to extract information out of it.


---
## Load the Output JSON response that was generated in the previous module

In [3]:
with open('lending-doc-output.json') as textract_json:
    data = json.load(textract_json)

data

{'DocumentMetadata': {'Pages': 7},
 'JobStatus': 'SUCCEEDED',
 'Results': [{'Page': 1,
   'PageClassification': {'PageType': [{'Value': 'PAYSLIPS',
      'Confidence': 99.99993896484375}],
    'PageNumber': [{'Value': 'undetected', 'Confidence': 100.0}]},
   'Extractions': [{'LendingDocument': {'LendingFields': [{'Type': 'PAY_PERIOD_START_DATE',
        'ValueDetections': []},
       {'Type': 'PAY_PERIOD_END_DATE',
        'ValueDetections': [{'Text': '7/18/2008',
          'Geometry': {'BoundingBox': {'Width': 0.07342242449522018,
            'Height': 0.009813770651817322,
            'Left': 0.7493308186531067,
            'Top': 0.24268266558647156},
           'Polygon': [{'X': 0.7493358254432678, 'Y': 0.24268266558647156},
            {'X': 0.8227532505989075, 'Y': 0.24279442429542542},
            {'X': 0.8227487802505493, 'Y': 0.2524964213371277},
            {'X': 0.7493308186531067, 'Y': 0.2523847818374634}]},
          'Confidence': 99.0}]},
       {'Type': 'PAY_DATE',
     

## Use the Textract Response Library to convert the response to an easier format to work with

The JSON is converted to a "flattened" CSV structure which is easier to work with. The CSV has the following columns:

- page_classification
- page_number_within_document_type
- page_number_in_document
- key
- key_confidence
- value
- value_confidence
- key-bounding-box.top
- key-bounding-box.height
- key-bb.width
- key-bb.left
- value-bounding-box.top
- value-bb.height
- value-bb.width
- value-bb.left


Here, we use the Textract Response Parser helper library:
[Link](https://pypi.org/project/amazon-textract-response-parser/)

In [4]:
trp2_doc: tl.TFullLendingDocument = tl.TFullLendingDocumentSchema().load(data)
lending_array = convert_lending_from_trp2(trp2_doc)

In [5]:
for row in lending_array:
    print(row)

['PAYSLIPS_undetected', '1', 'PAY_PERIOD_START_DATE', '1', '', '1', '0', '0', '0', '0', '0', '0', '0', '0']
['PAYSLIPS_undetected', '1', 'PAY_PERIOD_END_DATE', '1', '7/18/2008', '99.0', '0', '0', '0', '0', '0.24268266558647156', '0.009813770651817322', '0.07342242449522018', '0.7493308186531067']
['PAYSLIPS_undetected', '1', 'PAY_DATE', '1', '7/25/2008', '98.0', '0', '0', '0', '0', '0.2569517493247986', '0.009240644983947277', '0.07180937379598618', '0.7501305341720581']
['PAYSLIPS_undetected', '1', 'BORROWER_NAME', '1', 'JOHN STILES', '99.0', '0', '0', '0', '0', '0.31605130434036255', '0.009866639971733093', '0.1081274226307869', '0.5790428519248962']
['PAYSLIPS_undetected', '1', 'BORROWER_ADDRESS', '1', '101 MAIN STREET ANYTOWN, USA 12345', '92.0', '0', '0', '0', '0', '0.32689642906188965', '0.025985153391957283', '0.1985064595937729', '0.5790253281593323']
['PAYSLIPS_undetected', '1', 'COMPANY_NAME', '1', 'ANY COMPANY CORP.', '98.0', '0', '0', '0', '0', '0.24292130768299103', '0.010

We will use [Pandas Dataframes](https://pandas.pydata.org/) which is a widely used Python library in machine learning and data science. We will store the `lending_array` in a Pandas dataframe structure to further extract and analyze data according to the respective document labels/classifications.

In [6]:
import pandas as pd

df = pd.DataFrame(lending_array, columns = ['{page_classification}_{page_number_within_document_type}', 'page_number_in_document', 'key', 'key_confidence', 'value', 'value_confidence', 'key-bounding-box.top', 'key-bounding-box.height', 'key-bb.width', 'key-bb.left', 'value-bounding-box.top', 'value-bb.height', 'value-bb.width', 'value-bb.left'])
df

,{page_classification}_{page_number_within_document_type},page_number_in_document,key,key_confidence,value,value_confidence,key-bounding-box.top,key-bounding-box.height,key-bb.width,key-bb.left,value-bounding-box.top,value-bb.height,value-bb.width,value-bb.left
0,PAYSLIPS_undetected,1,PAY_PERIOD_START_DATE,1,,1,0,0,0,0,0,0,0,0
1,PAYSLIPS_undetected,1,PAY_PERIOD_END_DATE,1,7/18/2008,99.0,0,0,0,0,0.24268266558647156,0.009813770651817322,0.07342242449522018,0.7493308186531067
2,PAYSLIPS_undetected,1,PAY_DATE,1,7/25/2008,98.0,0,0,0,0,0.2569517493247986,0.009240644983947277,0.07180937379598618,0.7501305341720581
3,PAYSLIPS_undetected,1,BORROWER_NAME,1,JOHN STILES,99.0,0,0,0,0,0.31605130434036255,0.009866639971733093,0.1081274226307869,0.5790428519248962
4,PAYSLIPS_undetected,1,BORROWER_ADDRESS,1,"101 MAIN STREET ANYTOWN, USA 12345",92.0,0,0,0,0,0.32689642906188965,0.025985153391957283,0.1985064595937729,0.5790253281593323
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,W_2_undetected,6,LOCAL_INCOME_TAX_AMOUNT,1,$550.00,99.0,0,0,0,0,0.4061603248119354,0.014432545751333237,0.07119183242321014,0.7333780527114868
127,W_2_undetected,6,LOCALITY_NAME,1,Any Town,99.0,0,0,0,0,0.40753552317619324,0.00984314363449812,0.05929432436823845,0.8638736009597778
128,W_2_undetected,6,STATUTORY_EMPLOYEE,1,,1,0,0,0,0,0,0,0,0
129,W_2_undetected,6,RETIREMENT_PLAN,1,,1,0,0,0,0,0,0,0,0


Following are the document pages as they were classified in the previous document classification step: 

## 1. Payslip <a id="step1"></a>


### Get info (key-value) pairs from the response

In this section we will get the extractions in the document in key-value pair format.

In [7]:
payslip_df = pd.DataFrame()
payslip_df = df.loc[df['{page_classification}_{page_number_within_document_type}'] == "PAYSLIPS_undetected"].set_index('{page_classification}_{page_number_within_document_type}')

payslip_df[['key','value']]

,key,value
{page_classification}_{page_number_within_document_type},,
PAYSLIPS_undetected,PAY_PERIOD_START_DATE,
PAYSLIPS_undetected,PAY_PERIOD_END_DATE,7/18/2008
PAYSLIPS_undetected,PAY_DATE,7/25/2008
PAYSLIPS_undetected,BORROWER_NAME,JOHN STILES
PAYSLIPS_undetected,BORROWER_ADDRESS,"101 MAIN STREET ANYTOWN, USA 12345"
PAYSLIPS_undetected,COMPANY_NAME,ANY COMPANY CORP.
PAYSLIPS_undetected,COMPANY_ADDRESS,"475 ANY AVENUE ANYTOWN, USA 10101"
PAYSLIPS_undetected,FEDERAL_FILING_STATUS,Married
PAYSLIPS_undetected,STATE_FILING_STATUS,2


---
## 2. Check <a id="step2"></a>


### Get info (key-value) pairs from the response

In this section we will get the extractions in the document in key-value pair format.

> **_NOTE:_**  For this document, the following key-value pair included in the response: "SIGNATURE – SIGNATURE_PRESENT" indicates that a signature was found in the document.

In [8]:
checks_df = pd.DataFrame()
checks_df = df.loc[df['{page_classification}_{page_number_within_document_type}'] == "CHECKS_undetected"].set_index('{page_classification}_{page_number_within_document_type}')

checks_df[['key','value']]

,key,value
{page_classification}_{page_number_within_document_type},,
CHECKS_undetected,CHECK_NUMBER,0000000000
CHECKS_undetected,DATE,7/25/2008
CHECKS_undetected,CHECK_AMOUNT_IN_WORDS,TWO HUNDRED NINETY ONE AND 90/100 DOLLARS
CHECKS_undetected,DOLLAR_AMOUNT,$291.90
CHECKS_undetected,PAYEE_NAME,JOHN STILES
CHECKS_undetected,BORROWER_NAME,JOHN STILES
CHECKS_undetected,ACCOUNT_NUMBER,4040110157
CHECKS_undetected,BORROWER_ADDRESS,
CHECKS_undetected,BANK_ROUTING_NUMBER,122000496


---
## 3. Identity Document <a id="step3"></a>

### Get info (key-value) pairs from the response

In this section we will get the extractions in the document in key-value pair format.

In [9]:
ID_df = pd.DataFrame()
ID_df = df.loc[df['{page_classification}_{page_number_within_document_type}'] == "IDENTITY_DOCUMENT_undetected"].set_index('{page_classification}_{page_number_within_document_type}')

ID_df[['key','value']]

,key,value
{page_classification}_{page_number_within_document_type},,
IDENTITY_DOCUMENT_undetected,FIRST_NAME,GARCIA
IDENTITY_DOCUMENT_undetected,LAST_NAME,MARIA
IDENTITY_DOCUMENT_undetected,MIDDLE_NAME,
IDENTITY_DOCUMENT_undetected,SUFFIX,
IDENTITY_DOCUMENT_undetected,CITY_IN_ADDRESS,BIGTOWN
IDENTITY_DOCUMENT_undetected,ZIP_CODE_IN_ADDRESS,02801
IDENTITY_DOCUMENT_undetected,STATE_IN_ADDRESS,MA
IDENTITY_DOCUMENT_undetected,STATE_NAME,MASSACHUSETTS
IDENTITY_DOCUMENT_undetected,DOCUMENT_NUMBER,736HDV7874JSB


---
## 4. HomeOwners Insurance Application - _Amazon Textract Queries example_ <a id="step6"></a>

In document classification, this document was detected as `unclassified`, hence we will use Amazon Textract Queries feature to extract key information from such types of documents.

### Upload the unclassified document to S3

We will extract and upload the unclassified document to Amazon S3 Bucket. Note that documents will be uploaded into SageMaker's default S3 bucket. If you wish to use a different bucket please make sure you update the bucket name in `data_bucket` variable and also ensure that SageMaker has permissions to the S3 bucket.

In [10]:
import pdfplumber
input_pdf = "docs/lending_package.pdf"

with pdfplumber.open(input_pdf) as pdf:
    page7 = pdf.pages[6]
    image = page7.to_image(resolution=350).original
    image.save("docs/unclassified.pdf")

In [11]:
# Upload images to S3 bucket:
!aws s3 cp docs/unclassified.pdf s3://{data_bucket}/idp/textract/ --only-show-errors

In [12]:
file_key="idp/textract/unclassified.pdf"
display(IFrame(src=s3.generate_presigned_url('get_object', Params={'Bucket': data_bucket, 'Key': file_key}), width=600, height=700))

A we can see the homeowners insurance application is a document containing dense text. In this case we are interested in finding out a few key information from the entire document. Instead of extracting all the text from the document, and then apply logic (for example: regular expression) to find out that information, hence, we use the Amazon Textract Queries feature to grab the infromation from the document. 

Specifically, the information we are looking for are-

1. Named insured.
2. Policy number.
3. Effective date.
4. Insurance company.

We will craft questions in plain english language for the Textract API and pass it to the API call to get the information. Queries are-

1. Who is the named insured?
2. What is the policy number?
3. What is the effective date?
4. What is the insurance company name?

Also, we can see that all of this information is available in the single-page pdf document so we don't need the AI to look through all the pages to find this info. We will pass the page number when making the API call. Note: If the page number is not known then the `pages` parameter can be omitted.

In [13]:
from textractcaller import QueriesConfig, Query
from textractcaller.t_call import call_textract, Textract_Features

# Setup the queries
query1 = Query(text="Who is the named insured?" , alias="NAMED_INSURED", pages=["1"])
query2 = Query(text="What is the policy number?", alias="POLICY_NUMBER", pages=["1"])
query3 = Query(text="What is the effective date?", alias="EFFECTIVE_DATE", pages=["1"])
query4 = Query(text="What is the insurance company name?", alias="INSURANCE_COMPANY", pages=["1"])

#Setup the query config with the above queries
queries_config = QueriesConfig(queries=[query1, query2, query3, query4])

response_unclassified = call_textract(input_document=f's3://{data_bucket}/{file_key}',
                          features=[Textract_Features.QUERIES],
                          queries_config=queries_config)
doc_unclassified = Document(response_unclassified)

In [14]:
import trp.trp2 as t2
doc_unclassified : t2.TDocumentSchema = t2.TDocumentSchema().load(response_unclassified) 
    
entities = {}
for page in doc_unclassified.pages:
    query_answers = doc_unclassified.get_query_answers(page=page)
    if query_answers:
        for answer in query_answers:
            entities[answer[1]] = answer[2]
            
display(JSON(entities, root='Query Answers'))

<IPython.core.display.JSON object>

---
# Conclusion

In this notebook, we saw how to extract KEY-VALUE pairs from various documents that may be present in a mortgage packet. We also used Amazon Textract AnalyzeDocument Queries feature to extract specific information from an unclassified document, which is dense text and got accurate responses back from the API. In the next stages, we will continue with capabilities of Amazon Comprehend Custom NER and then perform enrichment on one of the documents.